In [ ]:
# Package Imports
import numpy as np
import sklearn
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# ---- CLASS IMBALANCE HANDLING AND STRATIFIED SPLIT ----

def rebalance(X, y):
  """ Function rebalances given dataset by oversampling the minority class to match the count of the majority class 

  Args:
      X (array): Feature Matrix
      y (array): Target Vector
  
  Returns:
      new_X (array): Rebalenced Feature Matrix
      new_y (array): Rebalenced Target Vector
  """
  
  # Getting Counts of Each Class and finding the size of the imbalence
  classes, counts = np.unique(y, return_counts = True)
  class_counts = dict(zip(classes, counts))
  minority_class = min(class_counts, key = class_counts.get)
  majority_class = max(class_counts, key = class_counts.get)
  imbalence = class_counts[majority_class] - class_counts[minority_class]
  
  # Finding Indices of Minority class
  minority_indices = np.where(y == minority_class)[0]
  
  # Randomly oversampling the required amount of points
  oversample = np.random.choice(minority_indices, size = imbalence)
  
  # Applying the oversampling to the feature matrix and target vector
  new_X = np.concatenate([X, X[oversample]], axis = 0)
  new_y = np.concatenate([y, y[oversample]], axis = 0)
    
  return new_X, new_y

def rebalanced_stratified_split(X_orig, y_orig, test_size = 0.2, random_state = None):
  """ Function performs a stratified split on the rebalanced dataset. First it rebalences dataset by calling
  'rebalance()' function then it performs the stratified split ensuring that class distribution is preserved in 
  both the train and test sets. 
  
  Args:
      X_orig (array): Original Feature Matrix 
      y_orig (array): Original Target Vector
      test_size (float, optional): Size of test set. Defaults to 0.2.
      random_state (int, optional): Seed for random number generation for reproducibility. Defaults to None.
    
  Returns: 
      X_train (array): Feature matrix for training
      X_test (array):Feature matrix for testing
      y_train (array): Target vector for training
      y_test (array): Target vector for testing
      """
      
  # Setting random seed
  np.random.seed(random_state)
  
  # First we rebalence the inputs
  X, y = rebalance(X_orig, y_orig)
  
  # Stratified split means we divide into test train while maintaining equal class distribution in both sets
  test_size = round(len(y) * test_size)
  train_size = len(y) - test_size
  
  # Assuming binary problem
  classes = np.unique(y, return_counts = False)
  neg, pos = classes[0], classes[1]
  
  # Finding Indices of Each Class
  p_indices = np.where(y == pos)[0]
  n_indices = np.where(y == neg)[0]
  
  # Randomly populating test set then finding the leftover train set each with equal class balence
  p_test_indices = np.random.choice(p_indices, size = int(np.ceil(test_size/2)))
  n_test_indices = np.random.choice(n_indices, size = int(np.ceil(test_size/2)))
  p_train_indices = np.setdiff1d(p_indices, p_test_indices)
  n_train_indices = np.setdiff1d(n_indices, n_test_indices)
  
  # Retreiving the test set
  X_test = np.concatenate([X[p_test_indices], X[n_test_indices]], axis = 0)
  y_test = np.concatenate([y[p_test_indices], y[n_test_indices]], axis = 0) 
  
  # Retreiving the train set
  X_train = np.concatenate([X[p_train_indices], X[n_train_indices]], axis = 0)
  y_train = np.concatenate([y[p_train_indices], y[n_train_indices]], axis = 0)
  
  # Re-shuffling the train and test feature matrix and target vector
  test_permutation = np.random.permutation(test_size)
  train_permutation = np.random.permutation(train_size)
  X_test = X_test[test_permutation]
  y_test = y_test[test_permutation]
  X_train = X_train[train_permutation]
  y_train = y_train[train_permutation]
  
  return X_train, X_test, y_train, y_test


In [ ]:
# ---- BASIC CLASSIFIER PERFORMANCE METRICS ----

def accuracy_score(y_true, y_pred):
  """ Calculates the accuracy of a classifier's predictions e.g. the ratio of correctly predicted instances to the
  total number of instances

  Args:
      y_true (array): Array of true labels
      y_pred (array): Array of predicted labels
    
  Returns:
      accuracy (float): Accuracy score
  """
  
  # Number of correct predictions
  correct_preds = np.sum(y_pred == y_true)
  
  # Total number of predictions
  total_preds = len(y_pred)
  
  # Ratio of correct preds to total preds i.e. accuracy score
  accuracy = correct_preds / total_preds
  
  return accuracy

def confusion_matrix(y_true, y_pred):
    """ Function computes the confusion matrix based on the true and predicted labels provided. Rows represent
    predicted labels and columns represent true labels.

    Args:
        y_true (array): Array of true labels
        y_pred (array): Array of predicted labels
    
    Returns:
        conf_matrix (array): Confusion Matrix
    """
    # Finding number of classes
    num_classes = len(set(y_true))
    
    # Initialising confusion matrix populated with zeros
    conf_matrix = np.zeros((num_classes, num_classes))
    
    # Populate confusion matrix with true on rows and predictions on columns similarly to coordinates
    for x, y in zip(y_true, y_pred):
        conf_matrix[x][y] += 1
        
    return conf_matrix

In [ ]:
# ---- LINEAR CLASSIFIER (LOGISTIC REGRESSION) OBJECT ----

class LinearClassifier:
    """ Class can fit a linear decision boundary to a given dataset and make predictions based on this boundary"""
    
    def __init__(self, threshold = 0.5):
        """ Constructor initialises the classifier object
        
        Args: 
            threshold (float): Detirmines the decision threshold. Default is 0.5
        """
        self.threshold = threshold
        self.weights = None
        self.lr = None
        self.epochs = None
        
    def fit(self, X, y):
        """ Method trains the linear classifier using the training data by estimating the weights (coefficients)
        and bias term of the decision boundary

        Args:
            X (array): Data Matrix
            y (array): Target Vector
        """
        # Initialising bias, weights, and sigmoid function
        ones = np.ones((X.shape[0], 1))
        X = np.concatenate((X, ones), axis = 1)
        self.weights = np.zeros(X.shape[1])
        sigmoid = lambda x: 1 / (1 + np.exp(-x))
        
        # Gradient Descent
        self.epochs = 1000
        self.lr = 0.01
        
        # For epochs
        for epoch in range(self.epochs):
            
            # Calculating predictions
            lm = np.dot(X, self.weights)
            preds = sigmoid(lm)
            
            # Computing error
            error = preds - y 
            
            # Computing gradients of loss function (w.r.t weights) by definition
            dw = (1 / X.shape[0]) * np.dot(X.T, error)
            
            # Updating weights
            self.weights -=  self.lr * dw
           
    def predict(self, X):
        """ Method predicts labels for the given inputs based on trained classifier (LM's outputs) considering the
        decision threshold specified during initialisation

        Args:
            X (array): Data Matrix
        """
        # Initialising bias, threshold and sigmoid function
        ones = np.ones((X.shape[0], 1))
        X = np.concatenate((X, ones), axis = 1)
        sigmoid = lambda x: 1 / (1 + np.exp(-x))
        threshold = self.threshold

        # Calculating predictions
        lm = np.dot(X, self.weights)
        probs = sigmoid(lm)
        
        # Getting predictions considering the threshold
        preds = (probs >= threshold).astype(int)
        
        return preds

In [ ]:
# ---- K-NEAREST NEIGHBOURS (KNN) CLASSIFIER AND KNN VARIANTS ----

class KNNClassifier:
  """ Here we implement a K-Nearest Neighbours Classifier which is capable of making predictions based on the majority
  class of the K nearest neighbours"""
  
  def __init__(self, k=3):
    """ Constructor method to initialise the classifier object. 
    
    Args: 
      k (int): Represents the number of neighbours to consider. Default is 3
    """
    self.k = k
  
  def fit(self, X, y):
    """ Fits the parameters of the model. 

    Args:
        X (array): Data Matrix
        y (array): Target Vector
    """
    self.X_train = X
    self.y_train = y
    
  def euclidean_distance(self, x1, x2):
    """ Calculates the Euclidean distance between two data points

    Args:
        x1 (array): Arbitrary Vector 
        x2 (array): Arbitrary Vector
    """
    return np.sqrt(np.sum(np.square(x1 - x2)))
    
  def predict_prob_single(self, x):
    """ Predicts the probability of each class label for a single data point based on its nearest neighbours in the dataset

    Args:
        x (array): Arbitrary Vector
    """
    # Finding distance between point and all other points using euclidean distance
    distances = np.zeros(len(self.X_train))
    for i, point in enumerate(self.X_train):
      distances[i] = self.euclidean_distance(x, point)
    
    # Finding k-nearest neighbours
    neighbours = np.argsort(distances)
    knn = neighbours[:self.k]
    knn_labels = self.y_train[knn]
  
    # Calculating probability of each class label
    unique_labels = np.unique(self.y_train, return_counts = False)
    counts = np.zeros(len(unique_labels))
  
    for label in knn_labels:
      counts[label] += 1
      
    probabilities = counts / self.k

    return probabilities
    
  def predict(self, X):
    """ Predicts the class labels for a given set of test data points based on the majority class of their nearest neighbours

    Args:
        X (array): Data Matrix
    """
    predictions = np.zeros(len(X))
    
    # We can equivalently find the class with the highest percentage 
    for i, point in enumerate(X):
      probs = self.predict_prob_single(point)
      max_prob = np.argmax(probs)
      predictions[i] = max_prob
    
    return predictions

class CostSensitiveKNNClassifier:
  """ This class implements a cost-sensitive knn classifier class which can make predictions based on the majority class of 
  the k-nearest neighbours while simultaneously considering class costs"""

  def __init__(self, k = 3, class_cost_matrix = [[0,1], [1,0]]):
    """ Constructor method to initialise classifier object

    Args:
        k (int): Represents the number of neighbours to consider. Default is 3
        class_cost_matrix (array): Cost Matrix. Default is [[0,1],[1,0]]
    """
    self.k = k
    self.cm = class_cost_matrix
      
  def fit(self, X, y):
    """ Method fits the parameters of the model

    Args:
        X (array): Data Matrix
        y (array): Target Vector
    """
    self.X_train = X
    self.y_train = y
    
  def euclidean_distance(self, x1, x2):
    """ Calculates the Euclidean distance between two data points

    Args:
        x1 (array): Arbitrary Vector 
        x2 (array): Arbitrary Vector
    """
    return np.sqrt(np.sum(np.square(x1 - x2)))
  
  def predict_prob_single(self, x):
    """ Predicts the probability of each class label for a sinlge data point based on its nearest neighbours in the dataset

    Args:
        x (array): Arbitrary Vector
    """
    # Finding distance between point and all other points using euclidean distance
    distances = np.zeros(len(self.X_train))
    for i, point in enumerate(self.X_train):
      distances[i] = self.euclidean_distance(x, point)
    
    # Finding k-nearest neighbours
    neighbours = np.argsort(distances)
    knn = neighbours[:self.k]
    knn_labels = self.y_train[knn]
  
    # Calculating probability of each class label
    unique_labels = np.unique(self.y_train, return_counts = False)
    counts = np.zeros(len(unique_labels))

    for label in knn_labels:
      counts[label] += 1
    
    probabilities = counts / self.k
    
    return probabilities

  def predict(self, X):
    """ Predicts the class labels for a given set of points based on class costs and class counts of nearest neighbours

    Args:
        X (array): Data Matrix
    """
    predictions = np.zeros(len(X))
    
    for i, point in enumerate(X):
      
      probs = self.predict_prob_single(point)
     
      # Costs of prediction: rows = predictions, columns = true i.e. 0,0 is cost of predicting 0 when actual is 0...
      ovr = probs @ self.cm
      
      # Finally finding neighbour with minimum cost
      max_ovr = np.argmin(ovr)

      predictions[i] = max_ovr
      
    return predictions

class GroupKNNClassifier:
    """ This class implements a Group weighted KNN classifier. Unlike other KNNs which treat all features equally, this class divides
    features into groups and assigns different weights to these groups. This allows the classifier to emphasize certain groups of features 
    over others which is useful when some features are known to be more informative"""
    
    def __init__(self, k, groups, group_weights):
      """ Constructor method initialises the classifier object 

      Args:
          k (int): Number of nearest neighbours to consider when making predictions
          groups (list): List of Lists where each sublist contains the indices of features belonging to a particular group
          group_weights (list): The weights asigned to each group
      """
      self.k = k
      self.groups = groups
      self.weights = group_weights
    
    def fit(self, X, y):
      """ Method fits the parameters of the model - Stores training data for use in prediction

        Args:
          X (array): Data Matrix
          y (array): Target Vector
      """
      self.X_train = X
      self.y_train = y
    
    def euclidean_distance(self, x1, x2):
      """ Calculates the Euclidean distance between two data points

      Args:
          x1 (array): Arbitrary Vector 
          x2 (array): Arbitrary Vector
      """
      return np.sqrt(np.sum(np.square(x1 - x2)))
    
    def distance(self, x1, x2):
      """ Method calculates the weighted distance between two data points considering the different groups of features and their 
      respective weights.

      Args:
          x1 (array): Arbitrary Vector
          x2 (array): Arbitrary Vector
      """
      
      weighted_distance = 0
      # Iterate through feature-weight pairs
      for features, weighting in zip(self.groups, self.weights):
        # Iterate  through features in group to handle case where there are multiple features
        for feature in features:
          # Calculates squared difference for current feature
          squared_difference = np.square(x1[feature] - x2[feature])
          # Applies corresponding weighting and adds to sum for whole loop
          weighted_distance += squared_difference * weighting
      
      # Square roots the sum to find final weighted difference 
      return np.sqrt(weighted_distance)

    def predict(self, X):
      """ Method predicts the class labels for a given set of test data points considering the group-weighted distances. 

      Args:
          X (array): Data Matrix
      """
      predictions = np.zeros(len(X))
      
      # Finding weighted distance between each point and all training points
      for i, point in enumerate(X):
        
        # Initialise array to store distances from current test point to all train points
        point_distances = np.zeros(len(self.X_train))
        for j, train_point in enumerate(self.X_train):
          # Calculates weighted distance
          point_distances[j] = self.distance(point, train_point)

        # Finds k-nearest neighbours w.r.t weighted distance
        neighbours = np.argsort(point_distances)
        knn = neighbours[:self.k]
        knn_labels = self.y_train[knn]
        
        # Calculating probability of each class label based on counts
        unique_labels = np.unique(self.y_train, return_counts = False)
        counts = np.zeros(len(unique_labels))
        
        for label in knn_labels:
          counts[label] += 1
        
        probabilities = counts / self.k
        
        # Populates prediction for current point with highest probability class label based on weighted nearest neighbours 
        predictions[i] = np.argmax(probabilities)
        
      return predictions

class AutoGroupsKNNClassifier:
    """ This class implements Auto-Group KNN classifier which automatically divides features into 2 groups (one for important features and one for less-important features) 
    and assigns weights to these groups to emphasize their importance. 
    """

    def __init__(self, k = 3, param = 0.8, weight = 0.2):
      """ Constructor method initialises the classifier object

      Args:
          k (int): Represents the number of neighbours to consider. Defaults to 3.
          param (float): Defines the threshold which defines the two groups. Defaults to 0.8.
          weight (float): Defines the weight that is given to less important features, and 1-weight is weight given to important features. Defaults to 0.2.
      """
      self.k = k 
      self.threshold = param
      self.groups = None
      self.weights = [(1 - weight), weight]
      
    
    def fit(self, X, y):
      """ Trains the classifier using given data and automatically selects feature groups based on the provided threshold

      Args:
          X (array): Data matrix
          y (array): Target Vector
      """
      # We have positive and negative data stacked on top of eachother within the informative and non-informative features so mean is useless
      # We chose to use standard deviation for our notion of importance
      feature_stds = np.std(X, axis = 0)

      # Finding index of features above and below threshold
      important_feature_indices = np.where(feature_stds > self.threshold)[0]
      unimportant_feature_indices = np.where(feature_stds <= self.threshold)[0]
      self.groups = [list(important_feature_indices), list(unimportant_feature_indices)]
      
      self.X_train = X
      self.y_train = y

    def predict(self, X):
      """ Predicts the class labels for given set of test data points using the trained classifier

      Args:
          X (array): Data matrix
      """
      predictions = np.zeros(len(X))
      
      # Finding weighted distance between each point and all training points
      for i, point in enumerate(X):
        
        # Initialise array to store distances from current test point to all train points
        point_distances = np.zeros(len(self.X_train))
        for j, train_point in enumerate(self.X_train):
          
          # Calculate weighted distance
          weighted_distance = 0
          # Iterate through feature-weight pairs
          for features, weighting in zip(self.groups, self.weights):
            # Iterate  through features in group to handle case where there are multiple features
            for feature in features:
              # Calculates squared difference for current feature
              squared_difference = np.square(point[feature] - train_point[feature])
              # Applies corresponding weighting and adds to sum for whole loop
              weighted_distance += squared_difference * weighting

          # Square roots the sum to find final weighted difference 
          point_distances[j] = np.sqrt(weighted_distance)

        # Finds k-nearest neighbours w.r.t weighted distance
        neighbours = np.argsort(point_distances)
        knn = neighbours[:self.k]
        knn_labels = self.y_train[knn]
        
        # Calculating probability of each class label based on counts
        unique_labels = np.unique(self.y_train, return_counts = False)
        counts = np.zeros(len(unique_labels))
        
        for label in knn_labels:
          counts[label] += 1
        
        probabilities = counts / self.k
        
        # Populates prediction for current point with highest probability class label based on weighted nearest neighbours 
        predictions[i] = np.argmax(probabilities)
        
      return predictions

In [ ]:
# ---- ADAPTIVE BOOSTING CLASSIFIER (ADABOOST) ----

def train_ab(X_train, y_train, param):
    """ Function trains an AdaBoost ensemble using decision tree classifiers as base learners. It iteratively updates the weights 
    of the training instances  based on their performance and combines multiple weak learners into a strong learner.

    Args:
        X_train (array): Training feature matrix
        y_train (array): Training labels
        param (int): Maximum depth of decision trees
    
    Returns: 
        model (list): List of tuples containing the trained/learned models and their corresponding weights  
    """
    n_instances = len(X_train)
    # Initialising weights arrays for training instances (uniform summing to one meaning equal probability) and learner, weight tuples to be appended too
    training_weights = np.ones(n_instances) / n_instances
    models_weights = []
    
    # We use 100 weak learners but this is down to preference
    for _ in range(100):
        
        # Training weak learner on weighted samples 
        weak_learner = DecisionTreeClassifier(max_depth = param)
        weak_learner.fit(X_train, y_train, training_weights)

        # Makes predictions
        preds = weak_learner.predict(X_train)
        
        # Calculating error
        accuracy = np.sum(training_weights[y_train == preds]) / np.sum(training_weights)
        error = 1.0 - accuracy

        # Calculates learner weighting by definition - called alpha in literature - added division by 0 handling  
        learner_weight = 0.5 * np.log((1 - error) / max(error, 1e-10))
        
        # Updates training weights by definition and re-normalises them
        training_weights = training_weights * np.exp(learner_weight * (preds != y_train))
        training_weights = training_weights / np.sum(training_weights)
        
        # Stores learner and weight
        models_weights.append((weak_learner, learner_weight))
    
    return models_weights

def test_ab(X_test, models):
    """ Function makes predictions on the test data using the trained AdaBoost ensemble model.

    Args:
        X_test (array): Test feature matrix
        models (list): List of tuples containing the learned models and their corresponding weights
    
    Returns: 
        predictions (array) Predicted labels for the test data
    """
    # Initialise predictions array
    preds = np.zeros((X_test.shape[0], len(models)))
    
    # For each model weight pairing
    for i, (learner, weight) in enumerate(models):
        
        # Makes weighted prediction
        current_pred = learner.predict(X_test)
        preds[:, i] = weight * current_pred
    
    # Extracts class
    predictions = np.sign(np.sum(preds, axis = 1)).astype(int)
    
    return predictions

class AdaBoostClassifier():
    # Class uses previously defined 'train_ab' and 'test_ab'
    def __init__(self, max_depth = 1):
        self.max_depth = max_depth
    def fit(self, X, y):
        self.models = train_ab(X, y, self.max_depth)
    def predict(self, X):
        return test_ab(X, self.models)

In [ ]:
# ---- RANDOM FOREST CLASSIFIER + BOOTSTRAPPING FUNCTIONS ----

def make_bootstrap(data_matrix, targets):
    """ Function generates bootstrapped replicate of the input dataset by sampling instances with replacement. The function then extracts the oob instances
    which are not included in the bootstrapped replicate.

    Args:
        data_matrix (array): Data matrix where each row represents an instance and each column represents a feature
        targets (array): Target vector containing labels corresponding to each instance in data matrix
    
    Returns:
        bootstrap_data_matrix (array): Bootstrapped replicate of input data matrix
        bootstrap_targets (array): Bootstrapped replicate of input target vector
        bootstrap_sample_ids (array): Vector containing the instance indices of boostrapped replicate of data matrix
        oob_data_matrix (array): Data matrix containing the out-of-bag instances
        oob_targets (array): Target Vector containing the labels of out-of-bag instances
        oob_samples_ids (array): Vector containing the instance indices of the out-of-bag instances
    """
    n_instances = len(data_matrix)
    
    # Generating random indices for bootstrapping
    bootstrap_sample_ids = np.random.choice(n_instances, n_instances, replace = True)

    # Creates bootstrapped replicate of data_matrix and targets using bootstrapping indices
    bootstrap_data_matrix = data_matrix[bootstrap_sample_ids]
    bootstrap_targets = targets[bootstrap_sample_ids]
    
    # Finding OOB sample ids as the ids which dont occur in bootstrap sample ids
    oob_samples_ids = np.setdiff1d(np.arange(n_instances), bootstrap_sample_ids)
    
    # Selecting OOB data
    oob_data_matrix = data_matrix[oob_samples_ids]
    oob_targets = targets[oob_samples_ids]
    
    return bootstrap_data_matrix, bootstrap_targets, bootstrap_sample_ids, oob_data_matrix, oob_targets, oob_samples_ids


def train_rfc(X_train, y_train, param):
    """ Function trains a random forest ensemble using 100 decision tree classifiers where each tree is trained on a bootstrapped sample of the training data

    Args:
        X_train (array): Training feature matrix
        y_train (array): Training labels
        param (int): Maximum depth of decision trees used in ensemble
    
    Returns: 
        random_forest (list): List of trained decision tree classifiers (the Random Forest Ensemble)
    """
    
    # Initialise list to store trained decision trees
    random_forest = []
    
    # We use 100 weak learners but this choice is just preference
    for _ in range(100):
        
        # Create bootstrapped sample of the training data (dont need the other outputs)
        sample, sample_targets, _, _, _, _ = make_bootstrap(X_train, y_train)
        
        # Trains decision tree and appends to random forest list
        tree = DecisionTreeClassifier(max_depth = param)
        tree.fit(sample, sample_targets) 
        random_forest.append(tree)
        
    return random_forest

def test_rfc(X_test, models):
    """ Function makes predictions on the test data using the trained Random Forest ensemble

    Args:
        X_test (array): Test feature matrix
        models (list): List of trained decision tree classifiers obtained from 'train_rfc'
    
    Returns: 
        predictions (array): Predicted labels for the test data
    """
    # Initialising array to store model predictions
    preds = np.zeros((len(models), X_test.shape[0]))
    
    # For each model
    for i, model in enumerate(models):
        # Predicts using model and stores prediction
        preds[i][:] = model.predict(X_test)
        
    # Calculates mean prediction along rows
    mean_preds = np.mean(preds, axis = 0)
    # Converts to binary predictions and converts 0 to -1 to work with other functions
    predictions = (mean_preds > 0).astype(int)
    predictions = np.where(predictions == 0, -1, predictions)

    return predictions

class RandomForestClassifier():
    # Class using previously defined 'train_rfc', 'test_rfc'
    def __init__(self, max_depth = 1):
        self.max_depth = max_depth
    def fit(self, X, y):
        self.models = train_rfc(X, y, self.max_depth)
    def predict(self, X):
        return test_rfc(X, self.models)

In [ ]:
# ---- OVO CLASSIFIER ----

def train_OvO(X_train, y_train, estimator):
    """ Function trains binary classifiers for all pairs of classes using specified base estimator
    
    Args:
        X_train (array): Feature matrix of shape (n_samples, n_features) 
        y_train (array): Labels of shape (n_samples)
        estimator (): The base estimator used for training the OvO classifier
  
    Returns:
        estimators (dict): Dictionary containing the trained binary classifiers for each pair of classes

    """
    # Randomly necessary for checkpoint at end of notebook
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    
    # Initialising dictionary and finding unique classes
    estimators = {}
    classes = sorted(set(y_train))
    
    # For all unique pairs (ignoring where i=j)
    for i in range(len(classes)):
        for j in range(i + 1, len(classes)):
            ci, cj = classes[i], classes[j]
            
            # Finding data and labels pertaining to current classes
            mask = np.logical_or(y_train == ci, y_train == cj)
            X = X_train.copy()
            y = y_train.copy()
            X = X[mask]
            y = y[mask]
  
            # Changing classes to {-1,1}
            yt = y.copy()
            yt[y == ci] = 1
            yt[y == cj] = -1
            
            # Training estimator for binary classification of current class pair
            est = copy.deepcopy(estimator)
            est.fit(X, yt)

            # Storing the trained estimator to dictionary
            estimators[(i, j)] = est

    return estimators

def test_OvO(X_test, estimators):
    """ Function predicts using trained binary estimators for all pairs of classes and combines results using majority voting

    Args:
        X_test (array): Feature matrix of shape (n_samples, n_features)
        estimators (dict): Dictionary containing trained binary classifiers for each pair of classes
    
    Returns:
        preds (array): Predicted labels for test data
    """
    # Randomly necessary for checkpoint at end of notebook
    X_test = np.array(X_test)
    
    # Initialising all predictions array
    unique_classes = set(class_index for class_pair in estimators.keys() for class_index in class_pair)
    scores = np.zeros((len(X_test), len(unique_classes)))
    
    # For each binary estimator
    for i, j in estimators: 
        est = estimators[(i, j)]
        
        # Uses binary classifier to predict between pair of classes
        pred = est.predict(X_test)
        
        # Need to ensure keep i,j - 1,-1 consistent
        scores[:,i][pred == 1] += 1
        scores[:,j][pred == -1] += 1
        
    preds = np.argmax(scores, axis = 1)
    return preds
   
class OVOClassifier():
    # Provides wrapper for training and testing the OvO classifier using the specified base estimator
    def __init__(self, estimator):
        self.estimator = estimator
    def fit(self, X, y):
        self.estimators = train_OvO(X, y, self.estimator)
    def predict(self, X):
        return test_OvO(X, self.estimators)